In [30]:
# - 売り上げ高
# - 営業利益
# まで
# かなりめんどくさい…

# todo 
# 営業活動キャッシュフロー
# 営業活動キャッシュフロー/売上=営業キャッシュフローマージン計算
# 営業キャッシュフローマージンをendDatetimeで降順ソートして直近5年(出せるとこまで、2年が限界か？)
# 算出し、15%以上が2年以上ならOperating_cash_flow_margin_listに銘柄を追加

In [31]:
import glob

from arelle.ModelValue import qname
from arelle import Cntlr
import numpy as np

import pandas as pd
pd.reset_option('display.max_rows')
print(pd.options.display.max_rows)
pd.reset_option('display.max_columns')
print(pd.options.display.max_columns)
import plotly.graph_objects as go

import stock_research as sr
taiouhyou = sr.taiouhyou


60
20


In [32]:
stock_code_list_path = '../warehouse/stock/created/submitted_doc_list_only_stock*.csv'
files = glob.glob(stock_code_list_path)
for file in files:
    print(file)

../warehouse/stock/created\submitted_doc_list_only_stock_20220301_20220428.csv
../warehouse/stock/created\submitted_doc_list_only_stock_20220330_20220428.csv


In [33]:
orig_data_path = '../warehouse/stock/original_data/'
stock_code_list = '../warehouse/stock/created/submitted_doc_list_only_stock_20220301_20220428.csv'
code_df = pd.read_csv(stock_code_list)

In [34]:
import warnings
warnings.filterwarnings('ignore')

In [35]:
 np.sort(code_df['secCode'].unique())[2:3]


array([1429], dtype=int64)

In [36]:
j=-1
# for stock_code in code_df['secCode'].unique():
# for stock_code in np.sort(code_df['secCode'].unique()):
# for stock_code in np.sort(code_df['secCode'].unique())[2:3]:
for stock_code in [1400]:

    j+=1
    if j==1:break
    df = []
    doc_id = code_df.query(f'secCode=={stock_code}').docID.values[0]
    date = code_df.query(f'secCode=={stock_code}').submitDateTime.values[0]
    company_name = code_df.query(f'secCode=={stock_code}').filerName.values[0]

    file_path = orig_data_path + date[:10] + '/' + doc_id + '/XBRL/PublicDoc/*.xbrl'
    file_path = glob.glob(file_path)[0]
    
    fact_df = sr.read_xbrl(file_path,stock_code)
    
    
    mask = fact_df.contextID.str.contains('NonConsolidated')
    mask = mask.astype(int)
    fact_df['NonConsolidated'] = mask
    
    # プロットに向かない数値以外のデータをすべて除外
    mask = fact_df['value'].str.match('^[0-9]+$')
    fact_df = fact_df[mask]
    
    for i,key in enumerate(taiouhyou):
        tmp = sr.get_key(fact_df,key)
        
        if i==0:
            df = tmp
        else:
            df = pd.concat([df,tmp],axis=0)
            
    df = sr.save_stock_statistics(df)


In [41]:
def make_operatingincome_margin(df):
    a = df.query('element_id == "NetSalesSummaryOfBusinessResults"').rename(columns={'value': 'value_sales'})
    a = a[['element_id','value_sales','endDatetime','NonConsolidated']]
    b = df.query('element_id == "NetCashProvidedByUsedInOperatingActivitiesSummaryOfBusinessResults"').rename(columns={'value': 'value_operatingincome'})
    b = b[['value_operatingincome','endDatetime','NonConsolidated']]
    tmp = pd.merge(a,b, on=['endDatetime','NonConsolidated'],how='left')
    tmp = sr.make_ratio(tmp,'value_operatingincome','value_sales','operatingincome_margin')
    tmp.drop('value_operatingincome',axis=1,inplace=True)

    df = pd.merge(df,tmp,on=['element_id','endDatetime','NonConsolidated'],how='left')
    df['operatingincome_margin'].replace(np.nan,'',inplace=True)
    
    return df

In [42]:
make_operatingincome_margin(df)

,element_id,value,contextID,startDatetime,endDatetime,instantDatetime,NonConsolidated,lag,Yoy,value_sales,operatingincome_margin
0,NetSalesSummaryOfBusinessResults,2.355196e+09,Prior4YearDuration,2017-01-01,2018-01-01,NaT,0,NaN,,2.355196e+09,
1,NetSalesSummaryOfBusinessResults,2.968591e+09,Prior3YearDuration,2018-01-01,2019-01-01,NaT,0,2.355196e+09,126%,2.968591e+09,
2,NetSalesSummaryOfBusinessResults,3.598007e+09,Prior2YearDuration,2019-01-01,2020-01-01,NaT,0,2.968591e+09,121%,3.598007e+09,21%
3,NetSalesSummaryOfBusinessResults,2.626680e+09,Prior1YearDuration,2020-01-01,2021-01-01,NaT,0,3.598007e+09,73%,2.626680e+09,12%
4,NetSalesSummaryOfBusinessResults,2.527792e+09,CurrentYearDuration,2021-01-01,2022-01-01,NaT,0,2.626680e+09,96%,2.527792e+09,25%
5,NetSalesSummaryOfBusinessResults,2.718420e+08,Prior4YearDuration_NonConsolidatedMember,2017-01-01,2018-01-01,NaT,1,NaN,,2.718420e+08,
6,NetSalesSummaryOfBusinessResults,7.101160e+08,Prior3YearDuration_NonConsolidatedMember,2018-01-01,2019-01-01,NaT,1,2.718420e+08,261%,7.101160e+08,
7,NetSalesSummaryOfBusinessResults,1.261467e+09,Prior2YearDuration_NonConsolidatedMember,2019-01-01,2020-01-01,NaT,1,7.101160e+08,177%,1.261467e+09,
8,NetSalesSummaryOfBusinessResults,2.459930e+08,Prior1YearDuration_NonConsolidatedMember,2020-01-01,2021-01-01,NaT,1,1.261467e+09,19%,2.459930e+08,
9,NetSalesSummaryOfBusinessResults,2.589230e+08,CurrentYearDuration_NonConsolidatedMember,2021-01-01,2022-01-01,NaT,1,2.459930e+08,105%,2.589230e+08,
